In [2]:
from math import ceil

import pandas as pd
import httpx
from httpx import AsyncClient
import asyncio
from flatten_json import flatten_json

In [4]:
states = {'qld' ,'wa','tas','nsw','vic','sa'}
states_dict = {}
for state in states:
  r  = httpx.get(f'https://www.urban.com.au/_next/data/7LtbXMYfk1smYiRsfTvzK/developments/{state}/page/1.json?slug=developments&slug={state}&slug=page&slug=1')
  states_dict[state] = r.json()['pageProps']['resource']['dataItems']['total']


In [5]:
async def get_data(client,state,i):
    r = await client.get((f'https://www.urban.com.au/_next/data/7LtbXMYfk1smYiRsfTvzK/developments/{state}/page/1.json?slug=developments&slug={state}&slug=page&slug={i}'))
    return r.json()

In [6]:
client  = httpx.AsyncClient(limits=httpx.Limits(max_connections=None, max_keepalive_connections=None,keepalive_expiry=None),timeout=None, http2=True)
responses  =await asyncio.gather(*[get_data(client,state,i) for state,count in states_dict.items() for i in range(1,ceil(count/41))])


In [7]:
urls = []
for reponse in responses:
  for item in reponse['pageProps']['resource']['dataItems']['items']:
    url = f'https://www.urban.com.au/_next/data/7LtbXMYfk1smYiRsfTvzK{item["url"]}.json'
    urls.append(url)

In [9]:
responses = await asyncio.gather(*[client.get(url) for url in urls])
flattened = []
for response in responses:
  flattened.append(flatten_json(response.json()))
await client.aclose()

In [10]:
df = pd.DataFrame(flattened)
df.to_csv('extracted houses.csv')